It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:


In [174]:
import pandas as pd
import numpy as np

In [175]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [176]:
df['FlightNumber'] = np.where(df['FlightNumber'].isnull(), df['FlightNumber'].shift(1)+10, df['FlightNumber'])
#np.where(df['FlightNumber'].isnull(), df['FlightNumber'].fillna(method="ffill")+10, df['FlightNumber'])

In [177]:
df['FlightNumber']

0    10045.0
1    10055.0
2    10065.0
3    10075.0
4    10085.0
Name: FlightNumber, dtype: float64

2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [178]:
df['From_To'].str.split("_").apply(lambda x:x[0]).str.title()

0      London
1      Madrid
2      London
3    Budapest
4    Brussels
Name: From_To, dtype: object

In [179]:
temp_df = pd.DataFrame()

In [180]:
temp_df['From'] = df['From_To'].str.split("_").apply(lambda x:x[0])
temp_df['To'] = df['From_To'].str.split("_").apply(lambda x:x[1])
temp_df

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)


In [181]:
temp_df['From'] = temp_df['From'].str.title()
temp_df['To'] = temp_df['To'].str.title()
temp_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions

In [182]:
df.columns

Index(['From_To', 'FlightNumber', 'RecentDelays', 'Airline'], dtype='object')

In [183]:
df.drop("From_To", axis=1, inplace=True)

In [184]:
df = pd.concat([df, temp_df], axis=1)

In [185]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,10055.0,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075.0,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


5.In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each 
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.

Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays`

In [186]:
df['RecentDelays']

0        [23, 47]
1              []
2    [24, 43, 87]
3            [13]
4        [67, 32]
Name: RecentDelays, dtype: object

In [187]:
df_indx = 0
deplays = pd.DataFrame()

In [188]:
def process_deplays(row):
    global df_indx
    for index, vale in enumerate(row):
        deplays.loc[df_indx, f'deplay_{index+1}'] = vale
    
    df_indx +=1    
    return df_indx    

In [189]:
df['RecentDelays'].apply(process_deplays)
deplays.reindex_like(df)
deplays

,deplay_1,deplay_2,deplay_3
0,23.0,47.0,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [190]:
#df.apply(lambda x: x['RecentDelays'], axis=1, result_type='expand')

In [191]:
df.rename({'RecentDelays':'delays'}, inplace=True, axis=1)

In [192]:
df = df.join(deplays, how='left')

In [193]:
df.sort_index(axis=1)

,Airline,FlightNumber,From,To,delays,deplay_1,deplay_2,deplay_3
0,KLM(!),10045.0,London,Paris,"[23, 47]",23.0,47.0,NaN
1,<Air France> (12),10055.0,Madrid,Milan,[],NaN,NaN,NaN
2,(British Airways. ),10065.0,London,Stockholm,"[24, 43, 87]",24.0,43.0,87.0
3,12. Air France,10075.0,Budapest,Paris,[13],13.0,NaN,NaN
4,"""Swiss Air""",10085.0,Brussels,London,"[67, 32]",67.0,32.0,NaN
